# Lyapunov Function Search

**Adapted from**: SOSTOOLS' SOSDEMO2 (See Section 4.2 of [SOSTOOLS User's Manual](http://sysos.eng.ox.ac.uk/sostools/sostools.pdf))

In [1]:
using DynamicPolynomials
@polyvar x[1:3]

(DynamicPolynomials.Variable{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}[x₁, x₂, x₃],)

We define below the vector field $\text{d}x/\text{d}t = f$

In [2]:
f = [-x[1]^3 - x[1] * x[3]^2,
     -x[2] - x[1]^2 * x[2],
     -x[3] - 3x[3] / (x[3]^2 + 1) + 3x[1]^2 * x[3]]

3-element Vector{MultivariatePolynomials.RationalPoly{DynamicPolynomials.Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}, Int64}, DynamicPolynomials.Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}, Int64}}}:
 (-x₁x₃² - x₁³) / (1)
 (-x₂ - x₁²x₂) / (1)
 (-4x₃ - x₃³ + 3x₁²x₃ + 3x₁²x₃³) / (1 + x₃²)

We need to pick an SDP solver, see [here](https://jump.dev/JuMP.jl/v1.12/installation/#Supported-solvers) for a list of the available choices.
We use `SOSModel` instead of `Model` to be able to use the `>=` syntax for Sum-of-Squares constraints.

In [3]:
using SumOfSquares
using CSDP
solver = optimizer_with_attributes(CSDP.Optimizer, MOI.Silent() => true)
model = SOSModel(solver);

We are searching for a Lyapunov function $V(x)$ with monomials $x_1^2$, $x_2^2$ and $x_3^2$.
We first define the monomials to be used for the Lyapunov function:

In [4]:
monos = x.^2

3-element Vector{DynamicPolynomials.Monomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, MultivariatePolynomials.Graded{MultivariatePolynomials.LexOrder}}}:
 x₁²
 x₂²
 x₃²

We now define the Lyapunov function as a polynomial decision variable with these monomials:

In [5]:
@variable(model, V, Poly(monos))

(_[1])·x₃² + (_[2])·x₂² + (_[3])·x₁²

We need to make sure that the Lyapunov function is strictly positive.
We can do this with a constraint $V(x) \ge \epsilon (x_1^2 + x_2^2 + x_3^2)$,
let's pick $\epsilon = 1$:

In [6]:
@constraint(model, V >= sum(x.^2))

(_[1] - 1)·x₃² + (_[2] - 1)·x₂² + (_[3] - 1)·x₁² is SOS

We now compute $\text{d}V/\text{d}x \cdot f$.

In [7]:
using LinearAlgebra # Needed for `dot`
dVdt = dot(differentiate(V, x), f)

((-8 _[1])x₃² + (-2 _[2])x₂² + (-2 _[1])x₃⁴ + (-2 _[2])x₂²x₃² + (-2 _[3] + 6 _[1])x₁²x₃² + (-2 _[2])x₁²x₂² + (-2 _[3])x₁⁴ + (-2 _[3] + 6 _[1])x₁²x₃⁴ + (-2 _[2])x₁²x₂²x₃² + (-2 _[3])x₁⁴x₃²) / (1 + x₃²)

The denominator is $x[3]^2 + 1$ is strictly positive so the sign of `dVdt` is the
same as the sign of its numerator.

In [8]:
P = dVdt.num

(-8 _[1])x₃² + (-2 _[2])x₂² + (-2 _[1])x₃⁴ + (-2 _[2])x₂²x₃² + (-2 _[3] + 6 _[1])x₁²x₃² + (-2 _[2])x₁²x₂² + (-2 _[3])x₁⁴ + (-2 _[3] + 6 _[1])x₁²x₃⁴ + (-2 _[2])x₁²x₂²x₃² + (-2 _[3])x₁⁴x₃²

Hence, we constrain this numerator to be nonnegative:

In [9]:
@constraint(model, P <= 0)

(8 _[1])·x₃² + (2 _[2])·x₂² + (2 _[1])·x₃⁴ + (2 _[2])·x₂²x₃² + (-6 _[1] + 2 _[3])·x₁²x₃² + (2 _[2])·x₁²x₂² + (2 _[3])·x₁⁴ + (-6 _[1] + 2 _[3])·x₁²x₃⁴ + (2 _[2])·x₁²x₂²x₃² + (2 _[3])·x₁⁴x₃² is SOS

The model is ready to be optimized by the solver:

In [10]:
JuMP.optimize!(model)

We verify that the solver has found a feasible solution:

In [11]:
JuMP.primal_status(model)

FEASIBLE_POINT::ResultStatusCode = 1

We can now obtain this feasible solution with:

In [12]:
value(V)

2.9958453255028186·x₃² + 12.286107321105156·x₂² + 15.718362431619155·x₁²

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*